<!-- from nltk import sent_tokenize
from gensim.utils import simple_preprocess

story = []

for filename in os.listdir('GOT'):
    with open(os.path.join('GOT', filename), encoding='latin-1') as file:
        corpus = file.read()
        raw_sent = sent_tokenize(corpus)
        for sent in raw_sent:
            story.append(simple_preprocess(sent)) -->

In [83]:
from nltk.corpus import stopwords
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import os

# nltk.download('punkt')
# nltk.download('stopwords')

stopwords_list = set(stopwords.words('english'))

story = []

for filename in os.listdir('GOT'):
    with open(os.path.join('GOT', filename), encoding='latin-1') as file:
        corpus = file.read()
        raw_sent = sent_tokenize(corpus)
        for sent in raw_sent:
            filtered_sent = [word for word in simple_preprocess(sent) if word.lower() not in stopwords_list]
            story.append(filtered_sent)

# story

In [84]:
len(story)

141219

In [63]:
from gensim.models import Word2Vec

model = Word2Vec(window=10, min_count=2, workers=4)
model.build_vocab(story)
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(4396920, 4579450)

In [64]:
model.wv.most_similar('daenerys')

[('stormborn', 0.9230729937553406),
 ('unburnt', 0.8967097401618958),
 ('viserys', 0.8319110870361328),
 ('targaryen', 0.8220386505126953),
 ('myrcella', 0.8171222805976868),
 ('queen', 0.81382155418396),
 ('elia', 0.8134568929672241),
 ('regent', 0.8066622018814087),
 ('dorne', 0.8051762580871582),
 ('loraq', 0.8040689826011658)]

In [65]:
model.wv.doesnt_match(['jon', 'rikon', 'robb', 'arya', 'sansa', 'bran'])

'jon'

In [66]:
model.wv.doesnt_match(['cersei', 'jaime', 'bronn', 'tyrion'])

'bronn'

In [67]:
model.wv['king'].shape

(100,)

In [68]:
model.wv.similarity('arya', 'sansa')

0.8037946

In [69]:
model.wv.similarity('cersei', 'sansa')

0.6809444

In [70]:
model.wv.similarity('tywin', 'sansa')

0.31701714

In [71]:
model.wv.get_normed_vectors()

array([[-0.07374443,  0.09640155,  0.17982861, ..., -0.09102281,
        -0.03991833,  0.01211485],
       [-0.21365058,  0.06534594,  0.23138341, ..., -0.04650346,
        -0.04907113, -0.00413792],
       [ 0.04215009,  0.08473474, -0.03237785, ..., -0.03394031,
         0.07174751, -0.18726595],
       ...,
       [ 0.00568629,  0.10225108,  0.11746372, ...,  0.11541904,
         0.10824168, -0.13204564],
       [-0.14559078,  0.05161754, -0.01633821, ..., -0.00885527,
        -0.01107212,  0.00291948],
       [-0.18378662,  0.13322134,  0.03311865, ..., -0.033397  ,
        -0.01969537, -0.00745804]], dtype=float32)

In [72]:
model.wv.get_normed_vectors().shape

(17312, 100)

In [74]:
y = model.wv.index_to_key

In [75]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X = pca.fit_transform(model.wv.get_normed_vectors())
X[:5]

array([[ 0.5086426 ,  0.01714688, -0.21909519],
       [ 0.38209596,  0.5639919 , -0.4028817 ],
       [ 0.5248525 , -0.03993364,  0.33437386],
       [-0.0323027 ,  0.15026616,  0.02700947],
       [ 0.2731328 ,  0.5105973 , -0.5933643 ]], dtype=float32)

In [76]:
X.shape

(17312, 3)

In [82]:
import plotly.express as plot

fig = plot.scatter_3d(X[:100], x=0, y=1, z=2, color=y[:100])
fig.show()